In [1]:
import json

import ee
import geemap
import geopandas as gpd
import shapely

In [2]:
ee.Initialize(project="ee-ursa-test")

In [3]:
bounds = json.loads(
    shapely.to_geojson(gpd.read_file("./bounds.gpkg")["geometry"].item()),
)

In [4]:
def filter_ds(ds_name: str, band_name: str) -> ee.Image:
    return (
        ee.ImageCollection(ds_name)
        .select(band_name)
        .filterBounds(bounds)
        .filterDate("2024-09-01", "2024-12-31")
        .reduce("mean")
        .clip(bounds)
    )


def plot_map(img: ee.Image, dmin: float, dmax: float, band_name: str):
    viz_options = {
        "min": dmin,
        "max": dmax,
        "bands": [f"{band_name}_mean"],
        "palette": ["black", "purple", "blue", "cyan", "green", "yellow", "red"],
        "opacity": 0.8,
    }

    m = geemap.Map(center=[31.7, -106.25], zoom=10)
    m.addLayer(img, viz_options)
    m.add_colorbar(viz_options)
    return m

# Aire

## CO2

In [7]:
img = filter_ds("COPERNICUS/S5P/OFFL/L3_CO", "CO_column_number_density")
geemap.download_ee_image(
    img,
    region=bounds,
    filename="co_mean.tif",
    scale=1000,
    crs="EPSG:4326",
)

co_mean.tif: |          | 0.00/56.4k (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


## Metano

In [28]:
band_name = "CH4_column_volume_mixing_ratio_dry_air"
img = filter_ds("COPERNICUS/S5P/OFFL/L3_CH4", band_name)
plot_map(img, dmin=1890, dmax=1950, band_name=band_name)

Map(center=[31.7, -106.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

## Ozono

In [29]:
band_name = "O3_column_number_density"
img = filter_ds("COPERNICUS/S5P/OFFL/L3_O3", band_name)
plot_map(img, dmin=0.1232, dmax=0.1245, band_name=band_name)

Map(center=[31.7, -106.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…